<a href="https://colab.research.google.com/github/lucas150/Barclays-Hack-O-Hire/blob/main/Hack_o_hire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#reading a CSV File

import pandas as pd
df=pd.read_csv('cyberbarclays.csv')
df

In [ ]:
#filling empty space with zero
df.fillna(0)

In [ ]:
#extracting date and time
import pandas as pd

df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d-%m-%Y %H:%M')

# Extract date, month, year, hour, and minute into separate columns
df['Date'] = df['Timestamp'].dt.day
df['Month'] = df['Timestamp'].dt.month
df['Year'] = df['Timestamp'].dt.year
df['Hour'] = df['Timestamp'].dt.hour
df['Minute'] = df['Timestamp'].dt.minute

df.drop(columns=['Timestamp'], inplace=True)

print(df.head())

In [ ]:
#extracting city and state
df[['City', 'State']] = df['location'].str.split(', ', expand=True)

df.drop(columns=['location'], inplace=True)

print(df.head())

In [ ]:
#droping columns
df.drop(columns=['State'], inplace=True)
df.drop(columns=['Source-IP'], inplace=True)
df.drop(columns=['Destination-IP'], inplace=True)
print(df.head())

In [ ]:
#reading updated file
df.to_csv('updated_cyber_data.csv', index=False)

In [ ]:
# Linear Regression Model 31% accuracy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score

# Read the data
df = pd.read_csv('updated_cyber_data.csv')

# Encode categorical variables using one-hot encoding
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_features = encoder.fit_transform(df[['Protocol', 'Anomaly', 'City']])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Protocol', 'Anomaly', 'City']))
df_encoded = pd.concat([df[['Date', 'Month', 'Year', 'Hour', 'Minute']], encoded_df], axis=1)

# Select features and target variables
X = df_encoded
y_severity = df['Severity']
y_attack = df['Attack']
y_anomaly = df['Anomaly']

# Encode target variables
label_encoder = LabelEncoder()
y_severity_encoded = label_encoder.fit_transform(y_severity)
y_attack_encoded = label_encoder.fit_transform(y_attack)
y_anomaly_encoded = label_encoder.fit_transform(y_anomaly)

# Split the data into training and testing sets
X_train, X_test, y_severity_train, y_severity_test = train_test_split(X, y_severity_encoded, test_size=0.2, random_state=42)
_, _, y_attack_train, y_attack_test = train_test_split(X, y_attack_encoded, test_size=0.2, random_state=42)
_, _, y_anomaly_train, y_anomaly_test = train_test_split(X, y_anomaly_encoded, test_size=0.2, random_state=42)

# Initialize the linear regression models with SGDRegressor for mini-batch training
severity_model = SGDRegressor()
attack_model = SGDRegressor()
anomaly_model = SGDRegressor()

# Define batch size
batch_size = 10000

# Train the linear regression models using mini-batch training
for i in range(0, len(X_train), batch_size):
    X_batch = X_train[i:i+batch_size]
    y_severity_batch = y_severity_train[i:i+batch_size]
    y_attack_batch = y_attack_train[i:i+batch_size]
    y_anomaly_batch = y_anomaly_train[i:i+batch_size]

    severity_model.partial_fit(X_batch, y_severity_batch)
    attack_model.partial_fit(X_batch, y_attack_batch)
    anomaly_model.partial_fit(X_batch, y_anomaly_batch)

# Make predictions
severity_predictions = severity_model.predict(X_test)
attack_predictions = attack_model.predict(X_test)
anomaly_predictions = anomaly_model.predict(X_test)

# Evaluate the models
severity_rmse = mean_squared_error(y_severity_test, severity_predictions, squared=False)
severity_r2 = r2_score(y_severity_test, severity_predictions)

attack_rmse = mean_squared_error(y_attack_test, attack_predictions, squared=False)
attack_r2 = r2_score(y_attack_test, attack_predictions)

anomaly_rmse = mean_squared_error(y_anomaly_test, anomaly_predictions, squared=False)
anomaly_r2 = r2_score(y_anomaly_test, anomaly_predictions)

print("Severity Model - RMSE:", severity_rmse, "R^2:", severity_r2)
print("Attack Model - RMSE:", attack_rmse, "R^2:", attack_r2)
print("Anomaly Model - RMSE:", anomaly_rmse, "R^2:", anomaly_r2)

# Calculate accuracy and precision for each model
threshold = 0.5

severity_class_predictions = (severity_predictions >= threshold).astype(int)
attack_class_predictions = (attack_predictions >= threshold).astype(int)
anomaly_class_predictions = (anomaly_predictions >= threshold).astype(int)

severity_accuracy = accuracy_score(y_severity_test, severity_class_predictions)
severity_precision = precision_score(y_severity_test, severity_class_predictions, average='weighted')

attack_accuracy = accuracy_score(y_attack_test, attack_class_predictions)
attack_precision = precision_score(y_attack_test, attack_class_predictions, average='weighted')

anomaly_accuracy = accuracy_score(y_anomaly_test, anomaly_class_predictions)
anomaly_precision = precision_score(y_anomaly_test, anomaly_class_predictions, average='weighted')

print("Severity Model - Accuracy:", severity_accuracy, "Precision:", severity_precision)
print("Attack Model - Accuracy:", attack_accuracy, "Precision:", attack_precision)
print("Anomaly Model - Accuracy:", anomaly_accuracy, "Precision:", anomaly_precision)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Severity Model - RMSE: 1770529819227624.8 R^2: -4.709650744461585e+30
Attack Model - RMSE: 2106968953691211.5 R^2: -6.72752161027069e+30
Anomaly Model - RMSE: 1154706560171712.2 R^2: -1.6534397315150553e+23
Severity Model - Accuracy: 0.334125 Precision: 0.111639515625
Attack Model - Accuracy: 0.340125 Precision: 0.11568501562500001
Anomaly Model - Accuracy: 0.000125 Precision: 1.5625e-08


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# SVR Model
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Read the data
df = pd.read_csv('updated_cyber_data.csv')

# Encode categorical variables using one-hot encoding
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_features = encoder.fit_transform(df[['Protocol', 'Anomaly', 'City']])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Protocol', 'Anomaly', 'City']))
df_encoded = pd.concat([df[['Date', 'Month', 'Year', 'Hour', 'Minute']], encoded_df], axis=1)

# Select features and target variables
X = df_encoded
y_severity = df['Severity']
y_attack = df['Attack']
y_anomaly = df['Anomaly']

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode the target variables
y_severity_encoded = label_encoder.fit_transform(y_severity)
y_attack_encoded = label_encoder.fit_transform(y_attack)
y_anomaly_encoded = label_encoder.fit_transform(y_anomaly)

# Split the data into training and testing sets for each target
X_train, X_test, y_severity_train_encoded, y_severity_test = train_test_split(X, y_severity_encoded, test_size=0.2, random_state=42)
_, _, y_attack_train_encoded, y_attack_test = train_test_split(X, y_attack_encoded, test_size=0.2, random_state=42)
_, _, y_anomaly_train_encoded, y_anomaly_test = train_test_split(X, y_anomaly_encoded, test_size=0.2, random_state=42)

# Initialize the SVR models with reduced parameters
severity_model = SVR(kernel='linear')
attack_model = SVR(kernel='linear')
anomaly_model = SVR(kernel='linear')

# Determine batch size
batch_size = 10000

# Train the SVR models in batches
for i in range(0, len(X_train), batch_size):
    X_batch = X_train[i:i+batch_size]
    y_severity_batch = y_severity_train_encoded[i:i+batch_size]
    y_attack_batch = y_attack_train_encoded[i:i+batch_size]
    y_anomaly_batch = y_anomaly_train_encoded[i:i+batch_size]

    severity_model.fit(X_batch, y_severity_batch)
    attack_model.fit(X_batch, y_attack_batch)
    anomaly_model.fit(X_batch, y_anomaly_batch)

# Make predictions
severity_predictions = severity_model.predict(X_test)
attack_predictions = attack_model.predict(X_test)
anomaly_predictions = anomaly_model.predict(X_test)

# Evaluate the models
severity_rmse = mean_squared_error(y_severity_test, severity_predictions, squared=False)
severity_r2 = r2_score(y_severity_test, severity_predictions)

attack_rmse = mean_squared_error(y_attack_test, attack_predictions, squared=False)
attack_r2 = r2_score(y_attack_test, attack_predictions)

anomaly_rmse = mean_squared_error(y_anomaly_test, anomaly_predictions, squared=False)
anomaly_r2 = r2_score(y_anomaly_test, anomaly_predictions)

print("Severity Model - RMSE:", severity_rmse, "R^2:", severity_r2)
print("Attack Model - RMSE:", attack_rmse, "R^2:", attack_r2)
print("Anomaly Model - RMSE:", anomaly_rmse, "R^2:", anomaly_r2)

In [ ]:
# Random forest Model 31% accuracy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load your dataset
df = pd.read_csv('updated_cyber_data.csv')

# Reduce the dataset size if necessary
sample_size = 10000
df_sample = df.sample(n=sample_size, random_state=42)

# Extract features and target variable
X = df_sample[['Protocol', 'Anomaly', 'Attack', 'Date', 'Month', 'Year', 'Hour', 'Minute', 'City']]
y = df_sample['Severity']  # Use 'Severity' as the target variable

# Convert categorical variables to numerical using one-hot encoding
X_encoded = pd.get_dummies(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Extract feature importances
feature_importances = rf_model.feature_importances_

# Print feature importances
for feature, importance in zip(X_encoded.columns, feature_importances):
    print(f"{feature}: {importance}")

In [ ]:
#updating Dataset
import pandas as pd
from datetime import datetime

# Load your dataset
df = pd.read_csv('updated_cyber_data.csv')

# Convert Date column to datetime format
# df['Date'] = pd.to_datetime(df['Date'])

# Combine Hour and Minute to create a new feature representing the exact time of the cyber attack
df['Attack_Time'] = df['Hour'].astype(str).str.zfill(2) + ':' + df['Minute'].astype(str).str.zfill(2)

# Create a feature that represents the day of the week based on the date
df['Day_of_Week'] = df['Date'].dt.day_name()

# Calculate the duration between the attack and the previous attack as a new feature
df['Previous_Attack_Date'] = df['Date'].shift(1)
df['Attack_Duration'] = (df['Date'] - df['Previous_Attack_Date']).dt.days

# Drop unnecessary columns if needed
df = df.drop(['Hour', 'Minute', 'Previous_Attack_Date'], axis=1)

# Save the modified dataset to a new CSV file
df.to_csv('modified_cyber_data.csv', index=False)

In [ ]:
#updating Dataset
import pandas as pd

# Load your dataset
df = pd.read_csv('updated_cyber_data.csv')

# Merge Date, Month, and Year into a single column
df['Date_Month_Year'] = df['Date'].astype(str) + '-' + df['Month'].astype(str) + '-' + df['Year'].astype(str)

# Drop Date, Month, and Year columns
df = df.drop(['Date', 'Month', 'Year'], axis=1)

# Save the modified dataset to a new CSV file
df.to_csv('modified_date_month_year.csv', index=False)

In [ ]:
#41% accuracy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load your dataset
df = pd.read_csv('modified_with_frequency.csv')

# Reduce the dataset size if necessary
sample_size = 10000
df_sample = df.sample(n=sample_size, random_state=42)

# Extract features and target variable
X = df_sample[['Protocol', 'Anomaly', 'Attack', 'Hour', 'Minute', 'City', 'Date_Month_Year', 'Day_of_Week', 'Frequency']]
y = df_sample['Severity']  # Use 'Severity' as the target variable

# Convert categorical variables to numerical using one-hot encoding
X_encoded = pd.get_dummies(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Extract feature importances
feature_importances = rf_model.feature_importances_

# Print feature importances
for feature, importance in zip(X_encoded.columns, feature_importances):
    print(f"{feature}: {importance}")

In [ ]:
#61% Accuracy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

# Load your dataset with the specified encoding
df = pd.read_csv('modified_with_frequency.csv', encoding='latin1')

# Reduce the dataset size if necessary
sample_size = 20000
df_sample = df.sample(n=sample_size, random_state=42)

# Extract features and target variable
X = df_sample[['Protocol', 'Anomaly', 'Attack', 'Hour', 'Minute', 'City', 'Date_Month_Year', 'Day_of_Week', 'Frequency']]
y = df_sample['Severity']  # Use 'Severity' as the target variable

# Perform one-hot encoding for categorical variables
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_encoded = encoder.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate and print precision
precision = precision_score(y_test, y_pred, average='weighted')  # Use weighted average for multi-class classification
print("Precision:", precision)

# Calculate and print confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)